In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Simulation parameters
n_players = 10
n_spins = 100
starting_wealth = 10000
bet_size = 100
p_win = 18/38  # American roulette probability
payout = 1  # 1:1 payout

# Generate wealth paths for players
np.random.seed(42)
player_wealths = np.zeros((n_players, n_spins + 1))
player_wealths[:, 0] = starting_wealth

casino_wealth = np.zeros(n_spins + 1)
casino_wealth[0] = 0

# Simulate spins
for i in range(n_spins):
    results = np.random.random(n_players) < p_win
    wins = results * bet_size * payout
    losses = ~results * bet_size
    net_results = wins - losses
    
    player_wealths[:, i+1] = player_wealths[:, i] + net_results
    casino_wealth[i+1] = casino_wealth[i] - sum(net_results)

# Find first zero for each player and set all future values to zero
for i in range(n_players):
    zero_indices = np.where(player_wealths[i] <= 0)[0]
    if len(zero_indices) > 0:
        first_zero = zero_indices[0]
        player_wealths[i, first_zero:] = 0

# Create figure with subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Player Wealth Paths', 'Casino Wealth Path')
)

# Add player traces with neon cyan and varying opacity
x_vals = np.arange(n_spins + 1)
for i in range(n_players):
    opacity = 0.3 + (0.7 * i/n_players)  # Varies from 0.3 to 1.0
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=player_wealths[i],
            mode='lines',
            line=dict(color=f'rgba(0, 255, 255, {opacity})', width=2),
            showlegend=False
        ),
        row=1, col=1
    )

# Add player wealth trendline
player_mean = np.mean(player_wealths, axis=0)
z = np.polyfit(x_vals, player_mean, 1)
p = np.poly1d(z)
fig.add_trace(
    go.Scatter(
        x=x_vals,
        y=p(x_vals),
        mode='lines',
        line=dict(color='red', width=2, dash='dash'),
        name='Player Trend',
        showlegend=False
    ),
    row=1, col=1
)

# Add casino trace
fig.add_trace(
    go.Scatter(
        x=x_vals,
        y=casino_wealth,
        mode='lines',
        line=dict(color='rgba(255, 215, 0, 1)', width=2),
        showlegend=False
    ),
    row=1, col=2
)

# Add casino trendline
z_casino = np.polyfit(x_vals, casino_wealth, 1)
p_casino = np.poly1d(z_casino)
fig.add_trace(
    go.Scatter(
        x=x_vals,
        y=p_casino(x_vals),
        mode='lines',
        line=dict(color='red', width=2, dash='dash'),
        showlegend=False
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    height=500,
    width=1100,
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(color='white')
)

# Update axes
for i in range(1, 3):
    fig.update_xaxes(
        title='Number of Spins',
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.2)',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='rgba(128,128,128,0.5)',
        row=1, col=i
    )
    fig.update_yaxes(
        title='Wealth ($)' if i == 1 else 'Casino Profit ($)',
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.2)',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='rgba(128,128,128,0.5)',
        row=1, col=i
    )

fig.show()

# Print slopes of trendlines
print(f"Player wealth trendline slope: ${z[0]:.2f} per spin")
print(f"Casino wealth trendline slope: ${z_casino[0]:.2f} per spin")

# Simulation parameters
n_players = 10
n_spins = 1000
starting_wealth = 10000
bet_size = 100

# Generate wealth paths for two groups of players
np.random.seed(42)

# Group 1: Game of Chance (negative EV)
chance_wealths = np.zeros((n_players, n_spins + 1))
chance_wealths[:, 0] = starting_wealth

# Group 2: Game with Optimal Action (positive EV) 
skilled_wealths = np.zeros((n_players, n_spins + 1))
skilled_wealths[:, 0] = starting_wealth

# Simulate spins
p_win_chance = 18/38  # Negative EV game
p_win_skilled = 0.55  # Positive EV game with optimal action

for i in range(n_spins):
    # Simulate Game of Chance results
    results_chance = np.random.random(n_players) < p_win_chance
    wins_chance = results_chance * bet_size
    losses_chance = ~results_chance * bet_size
    net_results_chance = wins_chance - losses_chance
    chance_wealths[:, i+1] = chance_wealths[:, i] + net_results_chance
    
    # Simulate Game with Optimal Action results
    results_skilled = np.random.random(n_players) < p_win_skilled
    wins_skilled = results_skilled * bet_size
    losses_skilled = ~results_skilled * bet_size
    net_results_skilled = wins_skilled - losses_skilled
    skilled_wealths[:, i+1] = skilled_wealths[:, i] + net_results_skilled

# Create figure with subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Players in a Game of Chance', 'Players in a Game of Incomplete Information with Optimal Action')
)

# Add traces for Game of Chance players
x_vals = np.arange(n_spins + 1)
for i in range(n_players):
    opacity = 0.3 + (0.7 * i/n_players)
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=chance_wealths[i],
            mode='lines',
            line=dict(color=f'rgba(0, 255, 255, {opacity})', width=2),
            showlegend=False
        ),
        row=1, col=1
    )

# Add Game of Chance trendline
chance_mean = np.mean(chance_wealths, axis=0)
z_chance = np.polyfit(x_vals, chance_mean, 1)
p_chance = np.poly1d(z_chance)
fig.add_trace(
    go.Scatter(
        x=x_vals,
        y=p_chance(x_vals),
        mode='lines',
        line=dict(color='red', width=2, dash='dash'),
        name='Trend',
        showlegend=False
    ),
    row=1, col=1
)

# Add traces for Skilled players
for i in range(n_players):
    opacity = 0.3 + (0.7 * i/n_players)
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=skilled_wealths[i],
            mode='lines',
            line=dict(color=f'rgba(0, 255, 0, {opacity})', width=2),
            showlegend=False
        ),
        row=1, col=2
    )

# Add Skilled players trendline
skilled_mean = np.mean(skilled_wealths, axis=0)
z_skilled = np.polyfit(x_vals, skilled_mean, 1)
p_skilled = np.poly1d(z_skilled)
fig.add_trace(
    go.Scatter(
        x=x_vals,
        y=p_skilled(x_vals),
        mode='lines',
        line=dict(color='red', width=2, dash='dash'),
        name='Trend',
        showlegend=False
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    height=500,
    width=1300,
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(color='white')
)

# Update axes
for i in range(1, 3):
    fig.update_xaxes(
        title='Number of Spins',
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.2)',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='rgba(128,128,128,0.5)',
        row=1, col=i
    )
    fig.update_yaxes(
        title='Wealth ($)',
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.2)',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='rgba(128,128,128,0.5)',
        row=1, col=i
    )

fig.show()

# Print slopes of trendlines
print(f"Game of Chance wealth trendline slope: ${z_chance[0]:.2f} per spin")
print(f"Game with Optimal Action wealth trendline slope: ${z_skilled[0]:.2f} per spin")


Player wealth trendline slope: $-4.88 per spin
Casino wealth trendline slope: $48.78 per spin


Game of Chance wealth trendline slope: $-4.25 per spin
Game with Optimal Action wealth trendline slope: $11.57 per spin
